参考链接：https://segmentfault.com/a/1190000044856508

### 基础数据验证 - 类型验证

In [11]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    username: str
    email: str

In [12]:
# 创建用户对象并进行基础数据验证  -> 验证数据类型
user_data = {"id": 111, "username": "john_doe", "email": "john.doe@example.com"}
# 错误数据 
# user_data = {"id": 111, "username": 666, "email": "john.doe@example.com"}
user = User(**user_data)  # 数据类型不匹配时会抛出异常
print(user)

id=111 username='john_doe' email='john.doe@example.com'


### 数据序列化 & 反序列化

In [13]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    username: str
    email: str

In [14]:
# todo pydantic库可以将数据序列化为JSON等格式
user_data = {"id": 1, "username": "john_doe", "email": "john.doe@example.com"}
user = User(**user_data)
# TODO 数据序列化
user.json()  #  等同于json.dumps()  

'{"id":1,"username":"john_doe","email":"john.doe@example.com"}'

In [15]:
user.dict()  #  json.loads()

{'id': 1, 'username': 'john_doe', 'email': 'john.doe@example.com'}

### 自定义数据校验器

In [33]:
from pydantic import BaseModel, constr, validator,field_validator

class User(BaseModel):
    username: str
    password: constr(min_length=8)

    # @validator('password')  --已弃用
    @field_validator('password')
    def validate_password(cls, value):
        # 校验是否包含 数字 isdigit()
        if not any(char.isdigit() for char in value):
            raise ValueError("Password must contain at least one digit")
        # 校验是否包含 字母 isalpha()
        if not any(char.isalpha() for char in value):
            raise ValueError("Password must contain at least one letter")
        return value

# 创建用户对象并进行数据验证
user_data = {"username": "john_doe", "password": "123a456789"}
user = User(**user_data)
print(user)

username='john_doe' password='123a456789'


In [31]:
eer = "12312"
any(ss.isalpha() for ss in eer)

False

In [27]:
[ss.isdigit() for ss in eer]

[True, True, True, True, True]

### 继承和扩展模型
pydantic库支持模型的继承和扩展，可以在现有模型基础上定义新的模型。
例如，定义一个管理员用户模型，继承自普通用户模型：

In [34]:
from pydantic import BaseModel

class User(BaseModel):
    username: str
    email: str

class AdminUser(User):
    role: str = "admin"

# 创建管理员用户对象并进行数据验证
admin_data = {"username": "admin_user", "email": "admin@example.com", "role": "admin"}
admin_user = AdminUser(**admin_data)
print(admin_user)

username='admin_user' email='admin@example.com' role='admin'


### 自定义序列化器和反序列化器
pydantic库允许开发者定义自定义序列化器和反序列化器，以实现特定的数据格式转换。

例如，定义一个自定义序列化器来将数据转换为XML格式：

In [50]:
from pydantic import BaseModel, root_validator, model_validator

class User(BaseModel):
    username: str
    email: str

    @root_validator(skip_on_failure=True)
    # @model_validator(mode='before')
    def to_xml(cls, values):
        xml_str = f"<user><username>{values['username']}</username><email>{values['email']}</email></user>"
        return {"xml_data": xml_str}

# 创建用户对象并进行数据转换
u_data = {"username": "john_doe", "email": "john.doe@example.com"}
user = User(**u_data)
print(user.xml_data)

<user><username>john_doe</username><email>john.doe@example.com</email></user>


C:\Users\un\AppData\Local\Temp\ipykernel_112052\3854746271.py:7: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @root_validator(skip_on_failure=True)
